# Crypto 2 - AES confusion

> I tried to implement an encrypt/decrypt service but it doesn't work. Could you help me figure why?
> 
> - ariana
>
> Files: [main.py](main.py)

This is a slightly disguised MT19937 cracking challenge. You generate a 16-byte key and 16-byte IV using `random.randbytes(16)`, and then you can do any of the following:
1. Encrypt as many messages as you want in CBC mode (with a new randomly generated IV each time).
2. Decrypt as many messages as you want in ECB mode. 
3. Get the flag encrypted using the original key/IV, at which point the program exits.

It is well-established that if you learn 624 consecutive 32-bit "reads" from an MT19937 that you know its entire state, so you can go forwards or backwards. Each key/IV makes up four reads here, so if we can learn 156 consecutive IVs we are done. It suffices for us to just send the empty message 156 times and decrypt them all, and we can learn the IVs this way.

First, here's a piece of code to guess predict a value given the next 624 values:

In [1]:
from z3 import *
def get_prev(arr):
    MT = [BitVec(f'm{i}', 32) for i in range(624)]
    s = Solver()
    
    def cache(x): # for some reason, this makes it faster in certain places
        tmp = Const(f'c{len(s.assertions())}', x.sort())
        s.add(tmp == x)
        return tmp
    
    def tamper(y):
        y ^= LShR(y, 11)
        y = cache(y ^ (y << 7) & 0x9D2C5680)
        y ^= cache((y << 15) & 0xEFC60000)
        return y ^ LShR(y, 18)
        
    def getnext():
        x = Concat(Extract(31, 31, MT[0]), Extract(30, 0, MT[1]))
        y = If(x & 1 == 0, BitVecVal(0, 32), 0x9908B0DF)
        MT.append(MT[397] ^ LShR(x, 1) ^ y)
        return tamper(MT.pop(0))
    
    getnext()
    prev = getnext()
    s.add([x == getnext() for x in arr])
    assert(s.check() == sat)
    return s.model().eval(prev).as_long()

# quick test case to check that it works
import random
testcase = [random.getrandbits(32) for _ in range(625)]
assert testcase[0] == get_prev(testcase[1:])

Now that we know it works, we can run our script!

In [2]:
from pwn import *
sh = process(['python', 'main.py'])
#sh = remote('34.124.157.94', 19522)
sh.sendlines([b'1\n'] * 156)

for _ in range(156):
    sh.readuntil(b'Ciphertext: ')
    sh.sendline(b'2')
    sh.send(sh.readline())

def get():
    sh.readuntil(b'Plaintext: ')
    return xor(bytes.fromhex(sh.readline().decode()), 16)

arr = [get() for _ in range(156)]
vals = [y for x in arr for y in struct.unpack('IIII', x)]

sh.sendline(b'3')
sh.readuntil(b'Flag: ')
flag = bytes.fromhex(sh.readline().decode())

for _ in range(8):
    vals.insert(0, get_prev(vals[:624]))
    
key = struct.pack('IIII', *vals[0:4])
iv = struct.pack('IIII', *vals[4:8])
print(f'{key.hex()=}')
print(f'{iv.hex()=}')

from Crypto.Cipher import AES
AES.new(key, AES.MODE_CBC, iv=iv).decrypt(flag)

[x] Starting local process '/usr/bin/python'
[+] Starting local process '/usr/bin/python': pid 14381
key.hex()='016a062bc396100fe53fa549ee72caf2'
iv.hex()='8ca2c8ea26860d51e3def57b1f31e060'


b'grey{<REDACTED>}\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'

The flag is `grey{tr4v3ll1n9_84ck_1n_t1m3}`.